# setup

In [ ]:
run_test=True
# run_test=False
DO_TRAIN_VOLS = True   # 是否运行 VOLS 的 VOL_GRID 训练
# DO_TRAIN_VOLS = False   # 是否运行 VOLS 的 VOL_GRID 训练
DO_TRAIN_PRCS = True  # 是否运行 PRCS 的 LMSE x VOL_GRID 交叉训练
# DO_TRAIN_PRCS = False  # 是否运行 PRCS 的 LMSE x VOL_GRID 交叉训练
Provisional=True
# Provisional=False


epochs=(1,1,1,0,100) if Provisional else (15,500,3,10,100)

cutdate='2025-11-18'
valData=''
use_amp=True
use_amp=False
# valData='2512'
testData=''
testData='new/'

weight=(0.81,1.01,0.03,0)
itr=5
metrics_cnt=6
import torch,sys

if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  mydrive='/content/drive/MyDrive/volrt/'
  checkpoints=mydrive+"checkpoints/"
  source_path=mydrive+"git/Crossformer"
  Provisional=Provisional and run_test
  tables = [ 'volvTSLA.csv', 'volvAAPL.csv'] if Provisional else [ 'volvTSLA.csv', 'volvAAPL.csv', 'volvGOOG.csv','volvNVDA.csv', 'volvMETA.csv', 'volvMSFT.csv', 'volvAMZN.csv', ]
  batch_size=int(torch.cuda.get_device_properties(0).total_memory/1e9+.5)*32
  import os,sys
  if not os.path.exists(source_path):
    %cd $mydrive/git
    !git clone https://github.com/jerronl/Crossformer.git
    %cd $source_path
  else:
    %cd $source_path
    # !git reset --hard HEAD
    # !git pull origin master
  %pip install einops #scikit-learn
  sys.path.append( source_path)
else:
  tables = [ 'volvAAPL.csv']
  testData=''
  mydrive= 'E:/mydoc/git/trade/analyics/'
  batch_size=32
  sys.path.append("G:/git/Crossformer1/")
  itr=1
  %load_ext autoreload
  %autoreload 2
%cd $mydrive


# Params

In [ ]:
# === Mode Settings ===
MODE = 'tuning'
# MODE = 'prod'

# Define Training Epochs based on Mode
# Note: Prod uses epochs[1] from the setup tuple
if MODE == 'tuning':
    TRAIN_EPOCHS = min(30,epochs[1])
else:
    TRAIN_EPOCHS = epochs[1]

# === Hyperparameter Grids ===
lambda_vars = [0.0004, 0.0005, 0.0005]
weight_overs = [1.4, 1.5]

# VOL_GRID: Used for 'vols' training and general grid search
# Format: (name, lambda_var, lambda_ce, lambda_shape, weight_over)
VOL_GRID = []
for lv in lambda_vars:
    for wo in weight_overs:
        name = f"LV{lv}_WO{wo}"
        VOL_GRID.append((name, lv, 2.0, 0.0, wo))

# LAMBDA_MSE_GRID: Used specifically for 'prcs' tuning if needed
LAMBDA_MSE_GRID = [
    ("lmse0.2", 0.2),
    ("lmse0.5", 0.5),
    ("lmse1.0", 1.0),
]
if Provisional:
  VOL_GRID = VOL_GRID[:2]
# Determine active grid slice based on epochs[3] (start) and epochs[4] (count)
start_idx = epochs[3]
end_idx = min(start_idx + epochs[4], len(VOL_GRID))
active_grid_indices = range(start_idx, end_idx)

print(f"Mode: {MODE} | Train Epochs: {TRAIN_EPOCHS}")
print(f"Active Grid Range: {start_idx} to {end_idx} (Total {len(active_grid_indices)} configs)")
for i in active_grid_indices:
    print(f"  [{i}] {VOL_GRID[i][0]}")

# Funcs

In [ ]:
from typing import Optional, Tuple
from cross_exp.exp_crossformer import Exp_crossformer
from data.data_loader import DatasetMTS, data_columns, data_names
from data.data_def import set_cat
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import seaborn as sns

# === Setup ===
set_cat(20)
from utils.tools import init_args, update_args

args = init_args()
data_parser = {}

# =============================================================================
# 1. 核心配置函数
# =============================================================================


# =============================================================================
# 2. 绘图与辅助函数 (修复了 IndexError)
# =============================================================================


def get_feature_names(ds_name, in_len):
    cols = data_columns(ds_name)
    return data_names(cols, 1)[0]


def prepare_preds_trues_for_plot(preds_raw, trues_raw, ycat=0):
    # 此函数将原始输出 (N, 20+5) 转换为用于绘制回归图的 (N, 5) 数组，并保留 IC 预测标签用于指标计算
    preds = np.asarray(preds_raw)
    trues = np.asarray(trues_raw)

    if ycat > 0 and preds.shape[1] > trues.shape[1]:
        # trues.shape[1] = 5
        logits = preds[:, :ycat]  # (N, 20)

        # 提取 5 个回归预测值 (IV_1 到 AUX)
        # 假设原始 preds 结构是 [Logits (20), IV_1_Pred, ..., AUX_Pred (5)]
        reg_preds = preds[:, ycat : ycat + trues.shape[1]].copy()  # (N, 5)

        # 计算预测标签
        logits_safe = np.where(np.isnan(logits), -np.inf, logits)
        cls_pred = (
            np.argmax(logits_safe, axis=1).astype(float) + 1.0
        )  # 假设真实标签是 1-20，预测标签也转换为 1-20

        # **将 IC 预测标签存储在第 5 列 (索引 4) 的位置**
        # 这一列是辅助列，在绘图循环中会被跳过，但其值可用于 i=0 时的 ACCR 计算
        reg_preds[:, 4] = cls_pred

        # reg_preds 现在是 [IV_1_Pred, IV_2_Pred, IV_3_Pred, IV_4_Pred, IC_Pred_Label]
        return reg_preds, trues

    return preds, trues


def _metrics_text(t, p):
    # 标准回归指标 (用于 i=1, 2, 3)
    diff = p - t
    mae = np.mean(np.abs(diff))
    mse = np.mean(diff**2)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs(diff) / (np.abs(t) + 1e-12))

    # ACCR 检查（此处应失败，但保留原始逻辑以防万一）
    accr = np.nan
    if (
        len(t) > 0
        and (np.unique(t).size <= 20)
        and (np.unique(t).size == np.unique(p).size)
    ):
        accr = np.mean((p.round() == t.round())[~np.isnan(p) & ~np.isnan(t)])

    if np.isnan(accr):
        return f"mae={mae:.4g}\nrmse={rmse:.4g}\nmape={mape:.3g}"
    else:
        return f"accr={accr:.3f}\nmae={mae:.4g}\nrmse={rmse:.4g}"


def _metrics_text_prcs_ic_iv(t_iv, p_iv, t_ic, p_ic_label):
    # 组合指标 (用于 i=0)
    # 1. 回归指标 (t_iv=trues[:, 0], p_iv=preds[:, 0])
    diff = p_iv - t_iv
    mae = np.mean(np.abs(diff))
    mse = np.mean(diff**2)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs(diff) / (np.abs(t_iv) + 1e-12))

    # 2. 分类指标 (t_ic=trues[:, 0], p_ic_label=preds[:, 4])
    accr = np.nan
    if len(t_ic) > 0 and (np.unique(t_ic).size <= 20) and (~np.isnan(t_ic)).sum() > 0:
        if np.unique(p_ic_label).size <= 20:
            # t_ic 是真实标签 (1-20)，p_ic_label 是预测标签 (1-20)
            accr = np.mean(
                (p_ic_label.round() == t_ic.round())[
                    ~np.isnan(p_ic_label) & ~np.isnan(t_ic)
                ]
            )

    if not np.isnan(accr):
        return f"accr(IC)={accr:.3f}\nmae(IV)={mae:.4g}\nrmse(IV)={rmse:.4g}"
    else:
        return f"mae(IV)={mae:.4g}\nrmse(IV)={rmse:.4g}\nmape(IV)={mape:.3g}"


def _compute_limits_like_old_regplot(trues_list, preds_list, feat_idx, feat_name):
    left, right = np.inf, -np.inf
    for trues, preds in zip(trues_list, preds_list):
        t = trues[:, feat_idx]
        p = preds[:, feat_idx]
        mask = ~np.isnan(t) & ~np.isnan(p)
        if not mask.any():
            continue
        t, p = t[mask], p[mask]
        if feat_name[:3] not in ["dtm", "pmc"]:
            left = min(left, max(np.min(t), -5), max(np.min(p), -5))
            right = max(right, min(np.max(t), 5), min(np.max(p), 5))
        else:
            left = min(left, np.min(t), np.min(p))
            right = max(right, np.max(t), np.max(p))
    if not np.isfinite(left) or not np.isfinite(right) or left == right:
        left, right = -1, 1
    return left, right


def plot_residual_heatmap_family(
    preds_raw,
    trues_raw,
    feature_names,
    title,
    bins=80,
    ycat=0,
    cols=4,
    figsize_per_cell=4,
):
    preds, trues = prepare_preds_trues_for_plot(preds_raw, trues_raw, ycat=ycat)
    out_dim = trues.shape[1]

    # 修复 IndexError: 确保绘图维度不超过特征名称列表的长度
    max_plot_dim = len(feature_names)
    effective_dim = min(out_dim, max_plot_dim)
    rows = int(np.ceil(effective_dim / cols))
    fig, axes = plt.subplots(
        rows,
        cols,
        figsize=(figsize_per_cell * cols, figsize_per_cell * rows),
        squeeze=False,
        constrained_layout=True,
    )

    for i in range(effective_dim):
        r, c = divmod(i, cols)
        ax = axes[r, c]
        fname = feature_names[i]

        # trues[:, i] 是 IV 真实值/IC 真实标签
        t = trues[:, i]
        # preds[:, i] 是 IV 预测值
        p = preds[:, i]
        mask = ~np.isnan(t) & ~np.isnan(p)
        t, p = t[mask], p[mask]

        if len(t) == 0:
            ax.set_title(fname)
            ax.axis("off")
            continue

        # --- [i=0 处，同时显示 IC/IV 指标] ---
        if ycat > 0 and i == 0:
            # Classification True Label: t (trues[:, 0])
            # Classification Pred Label: preds[:, 4] (IC_Pred_Label)
            p_cls = preds[mask, 4]
            metrics_text = _metrics_text_prcs_ic_iv(t, p, t, p_cls)
        else:
            # i=1, 2, 3 处，只显示回归指标
            metrics_text = _metrics_text(t, p)
        # --- [i=0 热力图现在可以绘制了] ---

        left, right = _compute_limits_like_old_regplot(
            [trues_raw], [preds_raw], i, fname
        )
        ax.hist2d(t, p, bins=bins, range=[[left, right], [left, right]], norm="log")
        ax.plot([left, right], [left, right], "w--", alpha=0.45)
        ax.set_xlim(left, right)
        ax.set_ylim(left, right)
        ax.set_xlabel("True")
        ax.set_ylabel("Pred")
        ax.set_title(fname, fontsize=12)
        ax.text(
            0.02,
            0.98,
            metrics_text,
            transform=ax.transAxes,
            va="top",
            ha="left",
            fontsize=9,
            bbox=dict(boxstyle="round,pad=0.25", fc="black", ec="none", alpha=0.35),
            color="white",
            zorder=3,
        )

    for i in range(effective_dim, rows * cols):
        axes.flatten()[i].axis("off")
    # 移除 suptitle 的 y 参数，依靠外部 print 语句创建间距
    if title:
        fig.suptitle(title, fontsize=14)
    plt.show()
    plt.close(fig)


def plot_multi_stock_residuals(
    all_preds_raw,
    all_trues_raw,
    stock_labels,
    feature_names,
    ycat=0,
    title="",
    bins=80,
    cols=4,
    figsize_per_cell=4,
):
    if not all_trues_raw:
        return
    out_dim = all_trues_raw[0].shape[1]

    # 修复 IndexError: 确保绘图维度不超过特征名称列表的长度
    max_plot_dim = len(feature_names)
    effective_dim = min(out_dim, max_plot_dim)
    rows = int(np.ceil(effective_dim / cols))
    import matplotlib.gridspec as gridspec

    fig = plt.figure(figsize=(figsize_per_cell * cols, figsize_per_cell * rows))
    outer = gridspec.GridSpec(rows, cols, wspace=0.25, hspace=0.35)
    cmap = plt.get_cmap("tab10") if len(stock_labels) <= 10 else plt.get_cmap("tab20")
    colors = [cmap(i) for i in range(len(stock_labels))]

    # 预处理所有数据，获取回归 preds 和 IC 预测标签
    all_preds_reg, all_trues_reg = [], []
    for preds_raw, trues_raw in zip(all_preds_raw, all_trues_raw):
        p_reg, t_reg = prepare_preds_trues_for_plot(preds_raw, trues_raw, ycat=ycat)
        all_preds_reg.append(p_reg)
        all_trues_reg.append(t_reg)

    for i in range(effective_dim):
        r, c = divmod(i, cols)
        ax_main = fig.add_subplot(outer[r, c])
        ax_hist_bg = ax_main.twinx()
        fname = feature_names[i]

        # 1. 聚合所有数据用于指标计算和轴限制
        combined_t, combined_p = all_trues_reg, all_preds_reg
        left, right = _compute_limits_like_old_regplot(combined_t, combined_p, i, fname)
        max_hist = 0

        for j, (preds_reg, trues_reg) in enumerate(zip(all_preds_reg, all_trues_reg)):
            t, p = trues_reg[:, i], preds_reg[:, i]
            mask = ~np.isnan(t) & ~np.isnan(p)
            t, p = t[mask], p[mask]

            # 2. 绘制散点图和背景直方图
            ax_main.scatter(
                t,
                p,
                s=5,
                alpha=0.4,
                label=stock_labels[j],
                color=colors[j % len(colors)],
                zorder=2,
            )
            if len(t) > 1:
                h, _, _ = ax_hist_bg.hist(
                    t,
                    bins=bins,
                    density=True,
                    alpha=0.15,
                    color=colors[j % len(colors)],
                    zorder=1,
                )
                max_hist = max(max_hist, np.max(h))

        # 3. 计算聚合指标
        all_t = np.concatenate([t[:, i] for t in combined_t])
        all_p = np.concatenate([p[:, i] for p in combined_p])
        mask = ~np.isnan(all_t) & ~np.isnan(all_p)
        t_all, p_all = all_t[mask], all_p[mask]

        if len(t_all) == 0:
            ax_main.set_title(fname)
            ax_main.axis("off")
            continue

        if ycat > 0 and i == 0:
            # Classification True Label: t_all (trues[:, 0])
            # Classification Pred Label: preds[:, 4]
            all_p_cls = np.concatenate([p[:, 4] for p in combined_p])
            p_cls_all = all_p_cls[mask]
            metrics_text = _metrics_text_prcs_ic_iv(t_all, p_all, t_all, p_cls_all)
        else:
            metrics_text = _metrics_text(t_all, p_all)

        # 4. 绘制轴线和文本
        ax_hist_bg.tick_params(
            axis="y", left=False, right=False, labelleft=False, labelright=False
        )
        ax_hist_bg.set_ylim(0, max_hist * 3.0 if max_hist > 0 else 1)
        ax_hist_bg.set_ylabel("")

        ax_main.plot([left, right], [left, right], "k--", alpha=0.45)
        ax_main.set_xlim(left, right)
        ax_main.set_ylim(left, right)
        ax_main.set_title(fname, fontsize=10)
        ax_main.text(
            0.02,
            0.98,
            metrics_text,
            transform=ax_main.transAxes,
            va="top",
            ha="left",
            fontsize=9,
            bbox=dict(boxstyle="round,pad=0.25", fc="black", ec="none", alpha=0.35),
            color="white",
            zorder=3,
        )
        ax_main.legend(fontsize=7, loc="lower right", framealpha=0.5)

    for i in range(effective_dim, rows * cols):
        fig.add_subplot(outer[i]).axis("off")
    # 移除 suptitle 的 y 参数，依靠外部 print 语句创建间距
    if title:
        fig.suptitle(title, fontsize=14)
    plt.tight_layout()
    plt.show()
    plt.close()


def plot_prcs_avg_prob_by_true_class(preds_raw, trues_raw, ycat, title=""):
    # 此函数接收原始 preds (包含 logits) 和 trues (包含真实标签)
    preds = np.asarray(preds_raw)
    trues = np.asarray(trues_raw)

    # 提取 logits
    logits = preds[:, :ycat]

    logits = np.where(np.isnan(logits), -np.inf, logits)
    probs = np.exp(logits - np.max(logits, axis=1, keepdims=True))
    probs = probs / (np.sum(probs, axis=1, keepdims=True) + 1e-12)

    # trues[:, 0] 是 IC label (1-20)
    true_cls = trues[:, 0].astype(int)
    x = np.arange(1, ycat + 1)
    fig, ax = plt.subplots(figsize=(6, 4))

    for k in range(1, ycat + 1):
        m = true_cls == k
        if m.any():
            ax.plot(x, probs[m].mean(axis=0), marker="o", label=f"True={k}", alpha=0.85)

    ax.set_title(title)
    ax.set_xlabel("Predicted Class")
    ax.set_ylabel("Average Probability")
    ax.legend(fontsize=8, loc="upper right")
    plt.show()


# =============================================================================
# 3. 配置与构建 Parsers & Training Functions (重构 train_models_for_volgrid)
# =============================================================================

COMMON_SETTINGS = {
    "e_layers": 5,
    "d_model": 512,
    "lradj": "type2",
    "checkpoints": checkpoints,
}


def build_parser(mode, ds, extra_query="", root_override=None):
    root = mydrive + valData if mode == "insample" else mydrive + testData
    if root_override:
        root = root_override
    q = (
        extra_query
        if mode == "insample"
        else (f"date>'#{cutdate}'" + (" and " + extra_query if extra_query else ""))
    )
    ds_config = COMMON_SETTINGS.copy()
    ds_config["root_path"] = root
    ds_config["data_path"] = tables
    ds_config["weight"] = weight[0] if ds == "vols" else weight[1]
    if q:
        ds_config["query"] = q
    parser = {}
    if ds != "vols":
        parser["vols"] = {"checkpoints": checkpoints}
    parser[ds] = ds_config
    return parser


def apply_volgrid_override(parser_config: dict, ds: str, grid_entry: list) -> dict:
    if ds in parser_config and isinstance(parser_config[ds], dict):
        temp_config = parser_config[ds].copy()
        is_full_parser = True
    else:
        temp_config = parser_config.copy()
        is_full_parser = False

    param_tag, lv, lce, lshape, wo = grid_entry
    temp_config["param_tag"] = param_tag
    temp_config["lambda_var"] = lv
    temp_config["lambda_ce"] = lce
    temp_config["lambda_shape"] = lshape
    temp_config["weight_overs"] = wo

    if is_full_parser:
        parser_config[ds] = temp_config
        return parser_config
    else:
        return temp_config


def build_train_parser(root_path=None):
    if root_path == None:
        root_path = mydrive + valData
    base = {
        "patience": epochs[0],
        "train_epochs": TRAIN_EPOCHS,
        "learning_rate": 0.01,
        "data_split": [0.7, 0.15, 0.15],
        "batch_size": batch_size * 2 // 5,
        "e_layers": 5,
        "d_model": 512,
        "lradj": "type2",
        "checkpoints": checkpoints,
        "root_path": root_path,
        "data_path": tables,
        "profile_mode": False,
        "use_amp": use_amp,
    }
    return {
        "vols": {**base, "data": "vols", "weight": weight[0], "over_weight": weight[2]},
        "prcs": {**base, "data": "prcs", "weight": weight[1], "over_weight": weight[3]},
    }


def train_models_for_volgrid(mode: str):
    """
    只运行 VOLS 的 VOL_GRID 训练。
    """
    global args, data_parser
    print(f"--- Starting VOLS_GRID Training (Mode: {mode}) ---")
    data_parser = build_train_parser(mydrive + valData)
    num_cycles = epochs[2]

    # 1. Train VOLS
    print(
        f"\n=== Starting VOLS Training ({len(active_grid_indices)} configs over {num_cycles} cycles) ==="
    )
    lr_vols = data_parser["vols"]["learning_rate"]
    SWEEP_KEYS_VOLS = ["lambda_ce", "lambda_shape", "over_weight"]

    for cycle in range(num_cycles):
        current_lr = lr_vols / (3.0**cycle)
        data_parser["vols"]["learning_rate"] = current_lr

        for p_idx in active_grid_indices:
            param_entry = VOL_GRID[p_idx]
            param_tag = param_entry[0]

            parser_v = apply_volgrid_override(
                data_parser["vols"].copy(), "vols", param_entry
            )
            setting_v = update_args(
                args, {"vols": parser_v}, 0, "vols", SWEEP_KEYS=SWEEP_KEYS_VOLS
            )

            exp_v = Exp_crossformer(args)
            # 打印当前参数组的下标
            print(
                f"[VOLS | Cycle {cycle+1}/{num_cycles} | IDX {p_idx} | {param_tag}] lr={current_lr:.6f} {setting_v}"
            )
            exp_v.train(setting_v, "vols")


def train_models_for_lmse_grid(mode: str):
    """
    运行 PRCS 的 LMSE_GRID x VOL_GRID 交叉训练。
    """
    if not LAMBDA_MSE_GRID or not VOL_GRID:
        print("Error: 训练网格未定义或为空。退出 LMSE grid 训练。")
        return

    data_parser = build_train_parser()
    num_cycles = epochs[2]
    num_lmse_settings = len(LAMBDA_MSE_GRID)
    lr_prcs = data_parser["prcs"]["learning_rate"]
    SWEEP_KEYS_PRCS_LMSE = ["lambda_ce", "lambda_shape", "over_weight", "lambda_mse"]

    print("\n--- Starting PRCS (LMSE + VOL) Cross-Grid Training (Idx >= 1000) ---")

    for cycle in range(num_cycles):
        current_lr = lr_prcs / (3.0**cycle)
        data_parser["prcs"]["learning_rate"] = current_lr

        for vol_idx_inner, vol_entry in enumerate(VOL_GRID):
            for lmse_idx, (lmse_tag, lambda_mse_value) in enumerate(LAMBDA_MSE_GRID):

                parser_p = data_parser["prcs"].copy()
                # 应用 VOL_GRID 参数
                parser_p = apply_volgrid_override(parser_p, "prcs", vol_entry)

                combined_tag = f"{parser_p['param_tag']}_{lmse_tag}"
                parser_p["param_tag"] = combined_tag
                # 应用 LMSE 参数
                parser_p["lambda_mse"] = lambda_mse_value

                unique_p_idx = (vol_idx_inner * num_lmse_settings) + lmse_idx + 1000
                full_parser = {"vols": {"checkpoints": checkpoints}, "prcs": parser_p}

                setting_p = update_args(
                    args, full_parser, 0, "prcs", SWEEP_KEYS=SWEEP_KEYS_PRCS_LMSE
                )
                exp_p = Exp_crossformer(args)

                # 打印当前参数组的下标
                print(
                    f"[PRCS(LMSE+VOL) | Cycle {cycle+1}/{num_cycles} | IDX {unique_p_idx} | Tag {combined_tag}] lr={current_lr:.6f}, L_MSE={lambda_mse_value:.4f}"
                )
                exp_p.train(setting_p, "prcs")


# =============================================================================
# 4. Testing Functions (添加了标题空行和 test_prcs_lmse_settings 汇总)
# =============================================================================


def run_test_one_setting(
    exp,
    setting_str,
    ds,
    dep_var,
    ycat,
    title_prefix,
    by_stock=False,
    stock_tables=None,
    plot_individual_heatmaps=True,
):
    metrics_list, all_preds, all_trues = [], [], []
    if by_stock:
        for tb in stock_tables:
            # exp.test returns raw preds and trues (inverse=True)
            preds, trues, m = exp.test(
                setting_str, ds, True, data_path=[tb], inverse=True
            )
            metrics_list.append(m)
            all_preds.append(preds)
            all_trues.append(trues)
            if plot_individual_heatmaps:
                # 添加空行和标题，满足用户要求
                print(f"\n--- Residual Heatmap: {title_prefix} {tb} ---")
                plot_residual_heatmap_family(
                    preds, trues, dep_var, title=f"{title_prefix} {tb}", ycat=ycat
                )
                if ds == "prcs":
                    print(f"\n--- Avg Prob Plot: {title_prefix} avg prob {tb} ---")
                    plot_prcs_avg_prob_by_true_class(
                        preds, trues, ycat=ycat, title=f"{title_prefix} avg prob {tb}"
                    )
        return metrics_list, all_preds, all_trues
    else:
        preds, trues, m = exp.test(setting_str, ds, True, inverse=True)
        metrics_list.append(m)
        # 添加空行和标题，满足用户要求
        print(f"\n--- Residual Heatmap: {title_prefix} ---")
        plot_residual_heatmap_family(
            preds, trues, dep_var, title=title_prefix, ycat=ycat
        )
        if ds == "prcs":
            print(f"\n--- Avg Prob Plot: {title_prefix} avg prob ---")
            plot_prcs_avg_prob_by_true_class(
                preds, trues, ycat=ycat, title=f"{title_prefix} avg prob"
            )
        return metrics_list, None, None


def generate_final_summary(summary_records):
    if not summary_records:
        print("No records to summarize.")
        return
    df = pd.DataFrame(summary_records)
    cols_order = [
        "exp",
        "dataset",
        "setting",
        "base_lr",
        "lambda_var",
        "weight_over",
        "lambda_ce",
        "lambda_shape",
        "lambda_mse",
        "mae",
        "mse",
        "rmse",
        "mape",
        "mdst",
        "accr",
        "epochs",
    ]
    print(df[[c for c in cols_order if c in df.columns]].to_string())


def test_all_volgrid_settings(
    mode: str, ds: str, by_stock: bool = True, plot_residuals: bool = True
):
    base_parser = build_parser(mode, ds)
    dep_var = get_feature_names(ds, getattr(args, "seq_len", 1))
    ycat = data_columns(ds).get("ycat", 0) if ds == "prcs" else 0
    summary_data = []
    print(
        f"\n--- Testing {ds} in {mode} ({len(active_grid_indices)} configurations) ---"
    )
    SWEEP_KEYS = []
    if ds == "vols":
        SWEEP_KEYS = ["lambda_ce", "lambda_shape", "over_weight"]

    for p_idx in active_grid_indices:
        ps = VOL_GRID[p_idx]
        param_name, lv, lce, lshape, wo = ps
        parser_config = apply_volgrid_override(base_parser.copy(), ds, ps)
        setting_str = update_args(args, parser_config, 0, ds, SWEEP_KEYS=SWEEP_KEYS)
        args.param_tag = param_name
        exp = Exp_crossformer(args)
        title = f"{ds} {mode} {param_name}"

        m_list, preds_fam, trues_fam = run_test_one_setting(
            exp,
            setting_str,
            ds,
            dep_var,
            ycat,
            title,
            by_stock=by_stock,
            stock_tables=tables,
            plot_individual_heatmaps=plot_residuals,
        )

        if by_stock and preds_fam and plot_residuals:
            # 绘制汇总图
            print(f"\n--- AGGREGATED RESIDUAL PLOT: {title} ---")
            plot_multi_stock_residuals(
                preds_fam,
                trues_fam,
                tables,
                dep_var,
                ycat=ycat,
                title=f"AGGREGATED {title}",
            )

        avg_metrics = np.mean(np.array(m_list), axis=0)
        rec = {
            "exp": param_name,
            "dataset": ds,
            "setting": setting_str,
            "base_lr": 0.01,
            "lambda_var": lv,
            "weight_over": wo,
            "lambda_ce": lce,
            "lambda_shape": lshape,
            "mae": avg_metrics[0],
            "mse": avg_metrics[1],
            "rmse": avg_metrics[2],
            "mape": avg_metrics[3],
            "mdst": avg_metrics[4],
            "accr": avg_metrics[5],
            "epochs": TRAIN_EPOCHS,
        }
        summary_data.append(rec)
    generate_final_summary(summary_data)
    return summary_data


def test_prcs_lmse_settings(
    mode: str, by_stock: bool = True, plot_residuals: bool = True
):
    ds = "prcs"
    dep_var = get_feature_names(ds, getattr(args, "seq_len", 1))
    ycat = data_columns(ds).get("ycat", 0)
    num_lmse_settings = len(LAMBDA_MSE_GRID)
    summary_data = []  # 初始化汇总列表
    print(f"\n--- Starting PRCS LMSE/VOL Cross Grid Testing ({mode}) ---")
    SWEEP_KEYS_PRCS_LMSE = ["lambda_ce", "lambda_shape", "over_weight", "lambda_mse"]

    for vol_idx_inner, vol_entry in enumerate(VOL_GRID):
        for lmse_idx, (lmse_tag, lambda_mse_value) in enumerate(LAMBDA_MSE_GRID):
            unique_p_idx = (vol_idx_inner * num_lmse_settings) + lmse_idx + 1000
            test_parser = build_parser(mode, ds)
            ds_config = apply_volgrid_override(test_parser[ds].copy(), ds, vol_entry)
            combined_tag = f"{ds_config['param_tag']}_{lmse_tag}"
            ds_config["param_tag"] = combined_tag
            ds_config["lambda_mse"] = lambda_mse_value
            test_parser[ds] = ds_config
            setting = update_args(
                args, test_parser, 0, ds, SWEEP_KEYS=SWEEP_KEYS_PRCS_LMSE
            )
            print(f"[TEST PRCS | Idx {unique_p_idx} | {combined_tag}]")
            exp = Exp_crossformer(args)

            # 运行测试并获取指标和数据
            m_list, preds_fam, trues_fam = run_test_one_setting(
                exp,
                setting,
                ds,
                dep_var,
                ycat,
                f"{mode} {combined_tag}",
                by_stock,
                tables,
                plot_residuals,
            )

            # 绘制汇总图
            if by_stock and preds_fam and plot_residuals:
                print(f"\n--- AGGREGATED RESIDUAL PLOT: {mode} {combined_tag} ---")
                plot_multi_stock_residuals(
                    preds_fam,
                    trues_fam,
                    tables,
                    dep_var,
                    ycat=ycat,
                    title=f"AGGREGATED {mode} {combined_tag}",
                )

            # 收集汇总数据
            avg_metrics = np.mean(np.array(m_list), axis=0)
            lv, lce, lshape, wo = vol_entry[1:]
            rec = {
                "exp": combined_tag,
                "dataset": ds,
                "setting": setting,
                "base_lr": 0.01,
                "lambda_var": lv,
                "weight_over": wo,
                "lambda_ce": lce,
                "lambda_shape": lshape,
                "lambda_mse": lambda_mse_value,
                "mae": avg_metrics[0],
                "mse": avg_metrics[1],
                "rmse": avg_metrics[2],
                "mape": avg_metrics[3],
                "mdst": avg_metrics[4],
                "accr": avg_metrics[5],
                "epochs": TRAIN_EPOCHS,
            }
            summary_data.append(rec)

    print(f"--- PRCS LMSE/VOL Cross Grid Testing Completed ({mode}) ---")
    generate_final_summary(summary_data)  # 输出汇总表格
    return summary_data


def test_vols_per_horizon(
    mode: str, param_entry: Optional[Tuple[str, float, float, float, float]] = None
):
    ds = "vols"
    dep_var = get_feature_names(ds, getattr(args, "seq_len", 1))
    horizons = [5, 10, 20]
    param_list = (
        [param_entry] if param_entry else [VOL_GRID[i] for i in active_grid_indices]
    )
    print(f"\n--- Running {mode} VOLS Per-Horizon Test (Horizons: {horizons}) ---")
    SWEEP_KEYS = ["lambda_ce", "lambda_shape", "over_weight"]

    for ps in param_list:
        param_name = ps[0]
        for h_len in horizons:
            base_parser = build_parser(mode, ds)
            parser = apply_volgrid_override(base_parser.copy(), ds, ps)
            hq = f"floor(horizon)=={h_len}"
            existing_query = parser[ds].get("query", "")
            parser[ds]["query"] = (existing_query + " and " + hq).lstrip(" and ")
            setting_str = update_args(args, parser, 0, ds, SWEEP_KEYS=SWEEP_KEYS)
            args.param_tag = param_name
            exp = Exp_crossformer(args)
            run_test_one_setting(
                exp,
                setting_str,
                ds,
                dep_var,
                0,
                title_prefix=f"{ds} {mode} {param_name} H{h_len}",
                by_stock=False,
                plot_individual_heatmaps=True,
            )


# train

In [ ]:
# --- [Train] ---
if run_test == False:
    # ... (其他初始化代码, 如 drive.mount) ...

    if DO_TRAIN_VOLS:
        print("\n--- [Train VOLS with VOL_GRID] ---")
        train_models_for_volgrid("insample") # 仅训练 VOLS

    if DO_TRAIN_PRCS:
        print("\n--- [Train PRCS with LMSE x VOL_GRID] ---")
        train_models_for_lmse_grid("insample") # 训练 PRCS

In [ ]:
if not run_test:
    from google.colab import runtime
    runtime.unassign()

In [ ]:
if not run_test:
    from cross_exp.exp_crossformer import Exp_crossformer
    from data.data_loader import DatasetMTS
    data_parser = {
        "vols": {
            'data':'vols',
            "patience":epochs[0],
            "train_epochs":epochs[1],
            'learning_rate':0.01,
            'data_split':[0.7,0.15,0.15],
            'batch_size':batch_size*2//5,
            'e_layers':5,
            'd_model':512,
            'lradj':'type2',
            "checkpoints":checkpoints,
            'root_path':mydrive+valData,
            'data_path':tables,
            'weight':weight[0],
            'over_weight':weight[2],
            'profile_mode':False,
            'use_amp':use_amp
        },
        "prcs":{
            'data':'prcs',
            'weight':weight[1],
            'over_weight':weight[3],
        }
        }
    s=epochs[3]
    for _ in range(epochs[2]):
        data_parser["vols"]["learning_rate"]=.01
        for i in range(epochs[2]):
            for ii in range(s,itr):
                # setting record of experiments
                # data_parser["vols"]['weight']=weight[0]
                setting=update_args(args,data_parser,ii)
                DatasetMTS.clear()
                # drive.mount('/content/drive/',force_remount=True)
                # !google-drive-ocamlfuse -cc # 注释掉这个bug行

                exp = Exp_crossformer(args)  # set experiments
                print(f"start training :lr={data_parser["vols"]["learning_rate"]} {setting}")
                exp.train(setting, "vols")

                print(f">>>>>>>testing : {setting}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
                preds, trues,_ = exp.test(setting, 'vols', True, data_path=[tables[-1]], inverse=True)
                print(preds.shape, trues.shape)

                # data_parser["vols"]['weight']=weight[1]
                setting=update_args(args,data_parser,ii,'prcs')
                DatasetMTS.clear()
                exp = Exp_crossformer(args)  # set experiments
                print(f"start training :lr={data_parser["vols"]["learning_rate"]} {setting}")
                exp.train(setting, "prcs")
                preds, trues,_ = exp.test(setting, 'prcs', True, data_path=[tables[-1]], inverse=True)
                print(preds.shape, trues.shape)
            data_parser["vols"]["learning_rate"]/=3.
            s=0


# test
##  PART A — In-sample (test split of same source)
Subparts:
1. vols per-stock heatmap  
2. prcs per-stock heatmap + avg prob by true class  
3. vols per-horizon heatmap


In [ ]:

if run_test:
            test_all_volgrid_settings(mode="insample", ds="vols", by_stock=True)

## In-sample - Prcs Per-Stock

Subparts:
1. vols per-stock heatmap  
2. prcs per-stock heatmap + avg prob by true class  
3. vols per-horizon heatmap


In [ ]:
if run_test:

            test_prcs_lmse_settings(mode="insample", by_stock=True)


## In-sample - Horizon


In [ ]:
if run_test:
        test_vols_per_horizon(mode="insample")

# PART B — Out-of-sample (testData, cutdate+)
## OOS - Vols Per-Stock

In [ ]:
if run_test:

        test_all_volgrid_settings(mode="oos", ds="vols", by_stock=True, plot_residuals=True)

## hlc

In [ ]:
if run_test:
        test_all_volgrid_settings(mode="oos", ds="prcs", by_stock=True)

## horizon


In [ ]:
if run_test:

        test_vols_per_horizon(mode="oos")

# end

In [ ]:
plt.show()
from google.colab import runtime
runtime.unassign()